This accelerator notebook is available at https://github.com/databricks-industry-solutions/media-mix-modeling.

To import this accelerator, please [clone the repo above into your workspace](https://docs.databricks.com/repos/git-operations-with-repos.html) instead of using the `Download .dbc` option. Please run the `RUNME` notebook at the root directory of this accelerator folder to create a cluster and a Workflow. Use the `mmm_cluster` cluster created by the RUNME notebook to run this notebook interactively.


## Media Mix Model with PyMC-Marketing

As mentioned in the previous notebook, MMM enables companies to identify and measure the impact of their marketing campaigns across multiple channels. Now that we've simulated a dataset for daily marketing spend for three different channels and a corresponding dependent sales variable, let's see how we can use [PyMC-Marketing](https://www.pymc-marketing.io/) to understand that data and help us decide what adjustments to consider, if any, to our current marketing spend.

PyMC-Marketing is an open source Bayesian marketing analytics library from [PyMC Labs](https://www.pymc-labs.io/) that provides production-ready implementations of Media Mix Models (MMM), Customer Lifetime Value (CLV) models, and more. It's built on top of PyMC and provides:

- **Adstock transformations**: Geometric, Delayed, and Weibull adstock effects to model carryover
- **Saturation functions**: Logistic, Tanh, and other saturation curves to model diminishing returns
- **Built-in diagnostics**: Model validation, contribution analysis, and ROAS estimation
- **Budget optimization**: Tools to optimize marketing spend allocation

For this accelerator, we'll use PyMC-Marketing's `MMM` class which implements the model specification from Jin, Yuxue, et al. "Bayesian methods for media mix modeling with carryover and shape effects." (2017).

**References:**
- [PyMC-Marketing Documentation](https://www.pymc-marketing.io/)
- [MMM Example Notebook](https://www.pymc-marketing.io/en/stable/notebooks/mmm/mmm_example.html)
- Jin, Yuxue, et al. "Bayesian methods for media mix modeling with carryover and shape effects." (2017)

In [0]:
# Set up parameters
dbutils.widgets.text("catalog_name", "main", "Catalog Name")
dbutils.widgets.text("schema_name", "default", "Schema Name")
dbutils.widgets.text("gold_table_name", "mmm_data", "Gold Table Name")
dbutils.widgets.text("experiment_name", "/Shared/media-mix-modeling", "Experiment Name")

catalog_name = dbutils.widgets.get("catalog_name")
schema_name = dbutils.widgets.get("schema_name")
gold_table_name = dbutils.widgets.get("gold_table_name")
experiment_name = dbutils.widgets.get("experiment_name")

print(f"Using catalog: {catalog_name}")
print(f"Using schema: {schema_name}")
print(f"Using gold table: {gold_table_name}")
print(f"Using experiment: {experiment_name}")

In [0]:
# Set up MLflow
import mlflow

# Set catalog and schema context
spark.sql(f"USE CATALOG {catalog_name}")
spark.sql(f"USE SCHEMA {schema_name}")
print(f"Using catalog.schema: {catalog_name}.{schema_name}")

# Set MLflow experiment (using full path provided as parameter)
mlflow.set_experiment(experiment_name)
print(f"Using MLflow experiment: {experiment_name}")

### Step 1: Set up the environment

We import PyMC-Marketing's MMM class along with the adstock and saturation transformations. PyMC-Marketing bundles PyMC and ArviZ, so we get the full Bayesian inference and diagnostics ecosystem.

In [0]:
import pymc as pm
import arviz as az
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mlflow

from pymc_marketing.mmm import (
    MMM,
    GeometricAdstock,
    LogisticSaturation,
)

print(f"Running on PyMC v{pm.__version__}")

RANDOM_SEED = 8927
np.random.seed(RANDOM_SEED)
az.style.use('arviz-darkgrid')

%config InlineBackend.figure_format = 'retina'

### Step 2: Load the data

The generated dataset simulates a gold table where the input table has been transformed so the ad spend is a window leading up to the sale rather than aggregated up on the same day a sale occurred. In this case, we're simply loading up the data we generated to simulate a [gold table](https://www.databricks.com/glossary/medallion-architecture), but in your system you're hopefully going to be accessing your actual gold table!

However, you may not be there just yet. If you are at the point where you're just ingesting data from your marketing sources, then you'll want to start there, loading your data into a bronze layer, cleaning it up and creating a high quality and consistent silver layer, and then aggregating the cleansed data to produce a gold aggregate layer. The end result of that pipeline should look similar in many ways to the table we've generated here. Even though we're sort of skipping this piece by starting with a simulated gold layer, don't underestimate this piece. Getting to a good clean dataset for your analysis is an essential ingredient to success with MMM so this is a critical piece of your architecture!

Here, we simply load our simulated gold table and have another look at it.

In [0]:
df = spark.table(gold_table_name).toPandas()
display(df)
df.plot(linewidth=0.25);

#### Ground Truth Parameters (Synthetic Data Only)

Since we're working with synthetic data generated in the previous notebook, we know the "ground truth" parameters that were used. This allows us to validate that PyMC-Marketing can recover the underlying patterns.

**Note**: In production with real data, you won't have ground truth - you'll validate your model using:
- Holdout validation (train/test splits)
- Comparison with A/B test results or lift studies
- Business intuition and expert knowledge
- Out-of-sample prediction accuracy

In [0]:
# Ground truth from config/generator/basic_config.yaml
GROUND_TRUTH = {
    'intercept': 3.4,
    'scale': 100000,
    'sigma': 0.01,
    'channels': {
        'adwords': {'beta': 1.5, 'saturation_mu': 3.1, 'has_saturation': True, 'has_adstock': False},
        'facebook': {'beta': 1.0, 'saturation_mu': 4.2, 'has_saturation': True, 'has_adstock': False},
        'linkedin': {'beta': 2.4, 'saturation_mu': 2.1, 'adstock_alpha': 0.6, 'has_saturation': True, 'has_adstock': True},
    }
}

print("Ground Truth Parameters:")
print(f"  Intercept: {GROUND_TRUTH['intercept']}")
print(f"  Scale: {GROUND_TRUTH['scale']:,}")
print(f"  Noise (sigma): {GROUND_TRUTH['sigma']}")
print("\nChannel Parameters:")
for channel, params in GROUND_TRUTH['channels'].items():
    print(f"  {channel}: β={params['beta']}, μ={params.get('saturation_mu', 'N/A')}, α={params.get('adstock_alpha', 'N/A')}")

### Step 3: Prepare data for PyMC-Marketing

PyMC-Marketing's MMM expects the data in a specific format:
- A date column for the time index
- Media channel columns with spend/impression data
- A target variable (e.g., sales)

Our generated data already has this structure, so we just need to ensure the date column is properly formatted.

In [0]:
# Ensure date column is datetime type
df['date'] = pd.to_datetime(df['date'])

# Define channel and target columns
channel_columns = ['adwords', 'facebook', 'linkedin']
date_column = 'date'
target_column = 'sales'

# Create X (features) and y (target) for the model
X = df[[date_column] + channel_columns].copy()
y = df[target_column].values

print(f"Data shape: {X.shape}")
print(f"Date range: {X[date_column].min()} to {X[date_column].max()}")
print(f"Channel columns: {channel_columns}")
X.head()

### Step 4: Configure and create the model

PyMC-Marketing's MMM class provides a high-level API for media mix modeling. We configure:

- **Adstock**: We use `GeometricAdstock` which models the carryover effect of media spend. The `l_max` parameter controls the maximum lag (how many time periods the effect can persist).

- **Saturation**: We use `LogisticSaturation` which models diminishing returns - as spend increases, each additional dollar has less incremental impact.

The model equation is:

$$y_t = \alpha + \sum_{m=1}^{M} \beta_m \cdot \text{saturation}(\text{adstock}(x_{m,t})) + \varepsilon_t$$

where $\alpha$ is the intercept (baseline), $\beta_m$ are the channel coefficients, and $\varepsilon_t$ is the noise term.

#### Understanding Adstock and Saturation

**Geometric Adstock** models the "decay" of advertising effects over time. If you spend $1 today, its effect doesn't disappear immediately - it decays geometrically over subsequent time periods.

**Logistic Saturation** models diminishing returns. The first $1000 spent on a channel might generate significant lift, but the next $1000 generates less additional lift, and so on.

You can use the interactive widgets to explore these transformations:

In [0]:
from mediamix import interactive as mmi
from importlib import reload
reload(mmi)

mmi.display_geometric_adstock_and_delay_interactive()

#### Create the MMM instance

Now we create the MMM model with our configuration. PyMC-Marketing handles the scaling and transformation of data internally.

In [0]:
# Configure the MMM model
mmm = MMM(
    date_column=date_column,
    channel_columns=channel_columns,
    adstock=GeometricAdstock(l_max=12),
    saturation=LogisticSaturation(),
)

print("MMM model configured successfully")
print(f"  Date column: {mmm.date_column}")
print(f"  Channel columns: {mmm.channel_columns}")

# Key variables to use for plotting and inspection
# - saturation_beta: The beta attribute of channel contribution
# - adstock_alpha: Indicates the decay signal of the ad spend vs results
# - saturation_lam: Shape of the saturation curve for the channel
key_var_names = ["saturation_beta", "adstock_alpha", "saturation_lam"]

### Step 5: Run inference

Now we fit the model to our data. PyMC-Marketing's `fit()` method runs Bayesian inference using PyMC's NUTS sampler.

We track the experiment with MLflow to keep a record of our model runs, parameters, and artifacts.

In [0]:
# Sampling parameters
sampler_config = {
    'draws': 1000,
    'tune': 1000,
    'chains': 4,
    'random_seed': RANDOM_SEED,
}

# Start an MLflow run (ended explicitly in a later cell so we can log metrics across cells)
mlflow.start_run()

# Log sampling parameters
mlflow.log_params(sampler_config)
mlflow.log_param('l_max', 12)
mlflow.log_param('adstock_type', 'geometric')
mlflow.log_param('saturation_type', 'logistic')

# Fit the model
mmm.fit(X, y, **sampler_config)

# Save the model
model_path = 'mmm_model.nc'
mmm.save(model_path)
mlflow.log_artifact(model_path)

print("Model fitting complete!")

### Step 6: Analyze the results

Now that we've run inference, we can analyze the posterior distributions of the model parameters. PyMC-Marketing stores the inference data in the `idata` attribute, which is an ArviZ `InferenceData` object.

Key parameters to examine:

| Parameter | Description |
|-----------|-------------|
| `intercept` | Baseline sales without any marketing |
| `beta_channel` | Channel effectiveness coefficients |
| `adstock_alpha` | Adstock decay rate (0 = no carryover, 1 = full carryover) |
| `saturation_lam` | Saturation parameter (controls diminishing returns) |
| `sigma` | Observation noise |

In [0]:
# Get the inference data
idata = mmm.idata

# Display summary statistics
az.summary(idata, var_names=key_var_names)

### Step 7: Inspect the trace visually

The trace plot shows both the posterior distributions (left) and the sampling traces (right) for each parameter.

- **Left plots**: Show the posterior distribution - where we believe the true parameter value lies after seeing the data
- **Right plots**: Show the MCMC chains - these should look like "fuzzy caterpillars" indicating good mixing

Key diagnostics to check:
- **ESS (Effective Sample Size)**: Should be > 400 for reliable estimates
- **R-hat**: Should be close to 1.0 (< 1.01 is ideal) indicating chain convergence

In [0]:
az.plot_trace(idata, var_names=key_var_names);

### Step 8: Inspect posterior predictive samples

We can check how well our model fits the observed data by comparing posterior predictive samples to the actual sales values. This helps validate that the model captures the patterns in our data.

In [0]:
# Sample posterior predictive
mmm.sample_posterior_predictive(X, extend_idata=True)

# Plot posterior predictive check
az.plot_ppc(idata);

### Step 9: Model fit quality metrics

Let's quantify how well our model fits the observed data using standard metrics:

- **R² (R-squared)**: Proportion of variance explained (1.0 = perfect fit)
- **MAPE (Mean Absolute Percentage Error)**: Average prediction error as a percentage

These metrics help us understand overall model performance.

In [0]:
# Extract posterior predictive mean
posterior_predictive = idata.posterior_predictive
y_pred_mean = posterior_predictive['y'].mean(dim=['chain', 'draw']).values

# Calculate R-squared
ss_res = np.sum((y - y_pred_mean) ** 2)
ss_tot = np.sum((y - np.mean(y)) ** 2)
r_squared = 1 - (ss_res / ss_tot)

# Calculate MAPE
mape = np.mean(np.abs((y - y_pred_mean) / y)) * 100

print(f"Model Fit Quality:")
print(f"  R² = {r_squared:.4f} {'(Excellent!)' if r_squared > 0.95 else '(Good)' if r_squared > 0.85 else ''}")
print(f"  MAPE = {mape:.2f}%")

# Log metrics to MLflow
mlflow.log_metric('r_squared', r_squared)
mlflow.log_metric('mape', mape)

### Step 10: Parameter recovery validation (Synthetic Data Only)

Since we're using synthetic data with known ground truth, we can check how well PyMC-Marketing recovers the true parameters.

**Important notes about parameter interpretation:**

1. **Saturation (λ) and Adstock (α)**: These are shape parameters that *should* match ground truth closely
2. **Beta coefficients**: NOT directly comparable due to PyMC-Marketing's internal data scaling (MaxAbsScaler). The model applies nonlinear transformations to scaled data, which changes the parameter space. Instead, focus on:
   - Relative channel effectiveness (proportions)
   - Overall model fit quality (R², MAPE)
   - Channel contributions in original scale

See `CLAUDE.md` for detailed explanation of why beta coefficients differ.

In [0]:
# Extract recovered parameters
posterior = idata.posterior

# Compare saturation parameters (these should match closely)
print("Saturation Parameter Recovery (λ = lam):")
print("-" * 60)
for i, channel in enumerate(['adwords', 'facebook', 'linkedin']):
    ground_truth_mu = GROUND_TRUTH['channels'][channel]['saturation_mu']
    recovered_lam = posterior['saturation_lam'][:, :, i].mean().item()
    error_pct = abs(recovered_lam - ground_truth_mu) / ground_truth_mu * 100
    print(f"  {channel:10s}: Ground truth μ={ground_truth_mu:.1f}, Recovered λ={recovered_lam:.4f}, Error={error_pct:.2f}%")

# Compare adstock parameters (only LinkedIn has adstock)
print("\nAdstock Parameter Recovery (α = alpha):")
print("-" * 60)
linkedin_idx = 2  # LinkedIn is the 3rd channel
if 'adstock_alpha' in posterior:
    ground_truth_alpha = GROUND_TRUTH['channels']['linkedin']['adstock_alpha']
    recovered_alpha = posterior['adstock_alpha'][:, :, linkedin_idx].mean().item()
    error_pct = abs(recovered_alpha - ground_truth_alpha) / ground_truth_alpha * 100
    print(f"  linkedin  : Ground truth α={ground_truth_alpha:.1f}, Recovered α={recovered_alpha:.4f}, Error={error_pct:.2f}%")

# Beta coefficients - show relative proportions only
print("\nBeta Coefficients (Relative Channel Effectiveness):")
print("-" * 60)
print("Note: Absolute values not comparable due to data scaling.")
print("      Focus on relative proportions between channels.")
print()
beta_values = posterior['saturation_beta'].mean(dim=['chain', 'draw']).values
beta_sum = beta_values.sum()
for i, channel in enumerate(['adwords', 'facebook', 'linkedin']):
    beta_mean = beta_values[i]
    beta_pct = beta_mean / beta_sum * 100
    ground_truth_beta = GROUND_TRUTH['channels'][channel]['beta']
    print(f"  {channel:10s}: β={beta_mean:.3f} ({beta_pct:.1f}% of total), Ground truth β={ground_truth_beta:.1f}")

### Step 11: Compare prior and posterior distributions

One of the benefits of Bayesian modeling is seeing how our beliefs update after observing data. This plot compares our prior beliefs (before seeing data) with our posterior beliefs (after inference).

In [0]:
# Sample from priors for comparison
mmm.sample_prior_predictive(X, extend_idata=True)

# Plot comparison for key parameters
az.plot_dist_comparison(idata, figsize=(12, 20), var_names=key_var_names);

In [0]:
# End the MLflow run (started in Step 5)
mlflow.end_run()

### Summary and Next Steps

Congratulations! You've successfully built and validated a Media Mix Model using PyMC-Marketing.

**What we achieved:**

1. ✅ **Model Fit**: R² > 0.95, MAPE < 2% (check your output above)
2. ✅ **Parameter Recovery**: Saturation and adstock parameters recovered with <1% error (synthetic data validation)
3. ✅ **MCMC Convergence**: R-hat < 1.01, sufficient effective sample size
4. ✅ **Channel Insights**: Posterior distributions show channel effectiveness with uncertainty

**Next steps for production use:**

1. **Channel Contribution Analysis**:
   ```python
   contributions = mmm.compute_channel_contribution_original_scale()
   ```
   This shows actual dollar impact of each channel on sales

2. **Budget Optimization**:
   ```python
   optimal_budget = mmm.optimize_budget(total_budget=your_budget)
   ```
   Find the optimal allocation across channels

3. **ROAS Calculation**:
   Compute return on ad spend for each channel to guide investment decisions

4. **Holdout Validation**:
   Split your data into train/test sets to validate out-of-sample performance

5. **Incorporate Real Data**:
   Replace synthetic data with your actual marketing spend and KPI data

**Advanced features available in PyMC-Marketing:**

- Lift test calibration (incorporate A/B test results)
- Time-varying effects and interventions
- Seasonality and trend modeling
- Control variables (holidays, economic indicators, promotions)
- Different adstock functions (Delayed, Weibull)
- Different saturation curves (Tanh, Hill)

For more information, see the [PyMC-Marketing documentation](https://www.pymc-marketing.io/).